In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [2]:
from pyngrok import ngrok

ngrok.set_auth_token("<auth_token>") # Intentionally hiding it

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import re

In [4]:
MODEL_NAME = "deepseek-ai/deepseek-coder-1.3b-instruct"

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

In [ ]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    device_map="auto"
)

In [7]:
# Core logic for detecting logical errors
def detect_errors(code: str, language: str):
    print(f"Code:\n{code}")
    prompt = f"""<|im_start|>system
    You are a {language} expert analyzing code for only logical errors.
    Detect only logical errors in the code snippet provided to you, enclosed within triple backticks (```).
    You don't have to detect any syntax errors. You don't have to detect any semantic errors. Only LOGICAL errors.
    The code starts from line number 1. Keep this in mind to return the response in the format mentioned below:
    Return JSON in the following format:
    {{
        "errors": [
            {{"lineNumber": int, "message": str}}
        ]
    }}
    If there are no logical errors, return an empty list, as follows:
    {{
        "errors": []
    }}
    <|im_end|>
    <|im_start|>user
    Analyze this {language} code: ```{code}```<|im_end|>
    <|im_start|>assistant
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=256,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [8]:
def extract_errors(decoded_response: str) -> dict:
    try:
        # Step 1: Isolate the assistant's response (everything after <|im_start|>assistant)
        assistant_part = decoded_response.split("<|im_start|>assistant")[-1]

        # Step 2: Extract the FIRST valid JSON block (strict match)
        json_match = re.search(r'\{[\s]*"errors"[\s]*:[\s]*\[[\s\S]*?\]\s*\}', assistant_part)

        if not json_match:
            return {"errors": []}  # No JSON found

        json_str = json_match.group()

        # Step 3: Parse and validate
        json_str = json_str.strip()
        result = json.loads(json_str)

        if not isinstance(result.get("errors", []), list):
            raise ValueError("'errors' must be a list")
        return result

    except Exception as e:
        return {"errors": [{"lineNumber": -1, "message": f"Error parsing JSON: {str(e)}"}]}

In [9]:
from fastapi import FastAPI, Request
import nest_asyncio
import uvicorn

app = FastAPI()
nest_asyncio.apply()

@app.post("/detect-logical-errors")
async def analyze_code(request: Request):
    data = await request.json()
    code = data.get("code")
    language = data.get("language", "python")

    decoded_response = detect_errors(code, language)
    result = extract_errors(decoded_response)
    errors_list = result["errors"]  # This is your list of errors!
    print(errors_list)
    return {"errors": errors_list}

In [ ]:
# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run FastAPI server (non-blocking)
uvicorn.run(app, host="0.0.0.0", port=8000)